In [22]:
import os
import sys
import uuid
import getpass
import numpy as np
import pandas as pd
import geopandas as gpd

user = getpass.getuser()

DVUTILS_LOCAL_CLONE_PATH = f"/Users/{user}/Documents/GitHub/dvutils"
sys.path.insert(0, DVUTILS_LOCAL_CLONE_PATH)
from utils_io import *

from arcgis import GIS
from gtfs_functions import Feed

## Pre-processing

In [23]:
# set working directory
dir = os.path.join("/Users", user, "Library", "CloudStorage", "Box-Box", "_GIS Analyses")

In [24]:
password = os.environ.get("AGOL_CONTENT_PASSWORD")
gis = GIS(url="https://mtc.maps.arcgis.com/home/", username="content_MTC", password=password)

In [25]:
# create a function to remove z values from a geometry
def remove_z(geom):
    """
    Removes z values from a geometry

    Source: https://gist.github.com/rmania/8c88377a5c902dfbc134795a7af538d8?permalink_comment_id=2893099#gistcomment-2893099
    """
    import shapely

    return shapely.wkb.loads(shapely.wkb.dumps(geom, output_dimension=2))

In [26]:
# create a function to read kml files by geometry type and return a geodataframe
def read_kml_by_geom_type(directory, geom_type):
    """Read a kml file by geometry type and return a geodataframe

    Parameters:
    -----------
    path : str
        path to the kml directory
    geom_type : str
        one of ['Point', 'LineString', 'Polygon']
    """
    import re
    import glob
    import fiona

    fiona.supported_drivers["KML"] = "rw"

    # use glob to get all the csv files
    # in the folder
    if geom_type == "Point":
        type_string = "_point"
    elif geom_type == "LineString":
        type_string = "_segment"
    else:
        type_string = "_polygon"

    pattern = re.compile(type_string)
    file_list = glob.glob(os.path.join(directory, "*.kml"))
    gdfs = list()

    for file in file_list:
        file_name = file.split("/")[-1]
        if pattern.search(file_name):
            # read the file
            gdf = gpd.read_file(file, driver="KML")
            # add the source file name
            gdf["source_file"] = file_name
            # remove z values
            gdf["geometry"] = gdf["geometry"].apply(remove_z)
            gdfs.append(gdf)
    gdf = pd.concat(gdfs, ignore_index=True)
    return gdf

In [27]:
# create a function to overwrite a feature layer
def overwrite_published_feature_layer(f_layer_id, geojson_path, client):
    """Overwrite a published feature layer

    Parameters:
    -----------
    f_layer_id : str
        id of the feature layer to overwrite
    geojson_path : str
        path to the geojson file
    client : authenticated arcgis client
        authentication example below:
        from arcgis.gis import GIS
        password = os.environ.get("AGOL_CONTENT_PASSWORD")
        gis = GIS(url="https://mtc.maps.arcgis.com/home/", username="content_MTC", password=password)
    """
    from arcgis.features import FeatureLayerCollection

    # get the feature layer
    host_flayer = client.content.get(f_layer_id)

    # create feature layer collection object
    f_layer = FeatureLayerCollection.fromitem(host_flayer)
    # overwrite the feature layer
    f_layer.manager.overwrite(geojson_path)

    print(f"Overwrote hosted feature layer with id: {f_layer_id}")

In [28]:
# create a function that publishes a geojson to agol
def publish_geojson_to_agol(
    geojson_path,
    layer_name,
    layer_snippet,
    tags,
    client,
    folder=None,
    overwrite=False,
    f_layer_id=None,
):
    """Publish a geojson to ArcGIS Online

    Parameters:
    -----------
    geojson_path : str
        path to the geojson file
    layer_name : str
        name of the layer
    layer_snippet : str
        layer snippet
    tags : list
        tags as a comma separated string (e.g. "tag1, tag2, tag3")
    client : authenticated arcgis client
        authentication example below:
        from arcgis.gis import GIS
        password = os.environ.get("AGOL_CONTENT_PASSWORD")
        gis = GIS(url="https://mtc.maps.arcgis.com/home/", username="content_MTC", password=password)
    folder : str
        name of the folder to publish to (optional)
    overwrite : bool
        if True, overwrite existing layer
    f_layer_id : str
        if overwrite is True, provide the id of the feature layer to overwrite
    """
    if overwrite:
        overwrite_published_feature_layer(f_layer_id, geojson_path, client)
    else:
        # publish the geojson
        item_prop = {
            "type": "GeoJson",
            "title": layer_name,
            "tags": tags,
            "snippet": layer_snippet,
            "overwrite": True,
        }
        item = client.content.add(item_properties=item_prop, data=geojson_path, folder=folder)

        # publish the item
        published_item = item.publish(file_type="geojson")

        print(f"Published {layer_name} to ArcGIS Online as {published_item.id}")

In [29]:
def create_flag_column(flag_gdf, original_gdf, original_id_col, out_column):
    if flag_gdf.shape[0] != original_gdf.shape[0]:
        original_gdf[out_column] = original_gdf[original_id_col].map(
            flag_gdf.groupby(original_id_col)[out_column].first()
        )
    else:
        original_gdf[out_column] = original_gdf[original_id_col].map(
            flag_gdf.set_index(original_id_col)[out_column]
        )

In [ ]:
def sort_list_values(row):
    row_list = list(row)
    row_list.sort()
    list_set = set(row_list)
    unique_list = list(list_set)
    return "; ".join(unique_list)

In [30]:
# read excel master list
hs_ms_df = pd.read_excel(
    os.path.join(dir, "BusAID Hotspot Master List_112823.xlsx"), sheet_name="hotspot_data_revised"
)

In [31]:
# drop rows with null id
hs_ms_df = hs_ms_df[~hs_ms_df["hotspot_id"].isnull()].copy()
# set id as integer
hs_ms_df["hotspot_id"] = hs_ms_df["hotspot_id"].astype(int)

In [32]:
# Remove the following hotspots from the analysis:
# - Remove hotspots #108, 109, 110 (WestCAT-identified hotspots) and any routes that only pass through these hotspots. These are being addressed via a DPD Forwards project.
# - Remove hotspots #82, 84, 85 (SFMTA-identified hotspots) and any routes that only pass through these hotspots. SFMTA has decided to withdraw these from consideration for the BusAID program.

hs_ms_df = hs_ms_df[~hs_ms_df["hotspot_id"].isin([108, 109, 110, 82, 84, 85])].copy()

In [33]:
# create point and line gdfs
point_gdf = read_kml_by_geom_type(os.path.join(dir, "Spatial Data"), "Point")
line_gdf = read_kml_by_geom_type(os.path.join(dir, "Spatial Data"), "LineString")

In [34]:
# extract the id from name string column and add to a new column
line_gdf["hotspot_id"] = line_gdf["Name"].str.extract(r"^\((\d+)\)", expand=False).astype(int)
point_gdf["hotspot_id"] = point_gdf["Name"].str.extract(r"^\((\d+)\)", expand=False).astype(int)

In [35]:
# merge master list with point and line gdfs
hs_point_gdf = pd.merge(point_gdf[["hotspot_id", "geometry"]], hs_ms_df, on="hotspot_id", how="inner")
hs_line_gdf = pd.merge(line_gdf[["hotspot_id", "geometry"]], hs_ms_df, on="hotspot_id", how="inner")

In [36]:
# read transit routes
# transit_url = "https://services3.arcgis.com/i2dkYWmb4wHvYPda/arcgis/rest/services/511_Transit_Routes_Sep_2023/FeatureServer/0"
# transit_gdf = pull_geotable_agol(base_url=transit_url, client=gis, reproject_to_analysis_crs=False)
api_key = os.environ.get("GTFS_API_KEY")
gtfs_path = (
    f"http://api.511.org/transit/datafeeds?api_key={api_key}&operator_id=RG&historic=2023-11"
)
feed = Feed(
    gtfs_path,
    time_windows=[0, 6, 10, 15, 19, 22, 24],
    busiest_date=False,
)

In [37]:
# create line frequency and route dataframes
# line_freq = feed.lines_freq
routes = feed.routes
agency = feed.agency
shapes = feed.shapes
trips = feed.trips

INFO:root:Reading "routes.txt".
INFO:root:Reading "agency.txt".
INFO:root:Reading "shapes.txt".
INFO:root:accessing trips
INFO:root:Start date is None. You should either specify a start date or set busiest_date to True.
INFO:root:Reading "trips.txt".
INFO:root:Reading "stop_times.txt".
INFO:root:_trips is defined in stop_times
INFO:root:Reading "stops.txt".
INFO:root:computing patterns


In [38]:
# join dataframes together to get route shapes with agency names
st_gdf = pd.merge(
    shapes, trips[["trip_id", "route_id", "direction_id", "shape_id"]], on="shape_id", how="inner"
)

str_gdf = pd.merge(
    st_gdf,
    routes[["route_id", "agency_id", "route_name", "route_type"]],
    on="route_id",
    how="inner",
)

stra_gdf = pd.merge(
    str_gdf,
    agency[["agency_id", "agency_name", "agency_url"]],
    on="agency_id",
    how="inner",
)

# reorder columns
cols = [
    "route_id",
    "agency_id",
    "trip_id",
    "direction_id",
    "shape_id",
    "agency_name",
    "route_name",
    "route_type",
    "agency_url",
    "geometry",
]
route_gdf = stra_gdf[cols].copy()

In [39]:
# # merge line frequency and route dataframes
# route_freq = pd.merge(
#     line_freq,
#     routes[["route_id", "agency_id", "route_short_name", "route_type"]],
#     on="route_id",
#     how="left",
# )

# # merge route frequency with agency
# route_freq = pd.merge(
#     route_freq, agency[["agency_id", "agency_name", "agency_url"]], on="agency_id", how="left"
# )

In [40]:
# check if all expected agency_ids are within the line_freq dataframe
expected_agency_ids = [
    "AC",
    "BA",
    "SR",
    "CC",
    "FS",
    "GG",
    "WH",
    "MA",
    "VN",
    "PE",
    "SM",
    "SF",
    "ST",
    "SO",
    "3D",
    "UC",
    "SC",
    "WC",
]
line_freq_agency_ids = route_gdf["agency_id"].unique().tolist()
missing_agency_ids = []
for id in expected_agency_ids:
    if id not in line_freq_agency_ids:
        missing_agency_ids.append(id)
missing_agency_ids

[]

In [41]:
# add human readable route type and direction
route_type_dict = route_type_dict = {
    0: "Tram, Streetcar, Light Rail",
    1: "Subway, Metro",
    2: "Rail",
    3: "Bus",
    4: "Ferry",
    5: "Cable Tram",
    6: "Aerial Lift",
    7: "Funicular",
    11: "Trollybus",
    12: "Monorail",
}
route_gdf["route_type_name"] = route_gdf["route_type"].map(route_type_dict)
route_gdf["direction_name"] = route_gdf["direction_id"].map({0: "Outbound", 1: "Inbound"})

In [42]:
# reorder columns
cols = [
    "route_id",
    "agency_id",
    "direction_id",
    "route_type",
    "agency_name",
    "route_name",
    # "route_short_name",
    "route_type_name",
    "direction_name",
    # "window",
    # "min_per_trip",
    # "ntrips",
    "agency_url",
    "geometry",
]
route_gdf = route_gdf[cols].copy()

In [43]:
# read epcs
epc_url = "https://services3.arcgis.com/i2dkYWmb4wHvYPda/arcgis/rest/services/communities_of_concern_2020_acs2018/FeatureServer/0"
epc_gdf = pull_geotable_agol(base_url=epc_url, client=gis, reproject_to_analysis_crs=False)

Breaking feature service layer IDs into 8 chunks


In [44]:
# read pdas
pda_url = "https://services3.arcgis.com/i2dkYWmb4wHvYPda/arcgis/rest/services/priority_development_areas_pba2050/FeatureServer/0"
pda_gdf = pull_geotable_agol(base_url=pda_url, client=gis, reproject_to_analysis_crs=False)

Breaking feature service layer IDs into 1 chunks


In [45]:
# filter transit route data to only include routes with the following conditions:
# - route_type = 3 (bus)
# - window = "6:00-10:00" (AM peak)
# - dir_id = "Inbound"

# bus_gdf = transit_gdf.query("route_type == 3 and window == '6:00-10:00' and dir_id == 'Inbound'").copy().to_crs("EPSG:26910")
bus_lr_gdf = (
    route_gdf.query("route_type_name.isin(['Bus', 'Tram, Streetcar, Light Rail'])")
    .copy()
    .to_crs("EPSG:26910")
)

In [46]:
# create a dataframe of hotspots exploding comma separated route ids into multiple rows
hs_route_df = hs_ms_df[["hotspot_id", "agency", "transit_routes"]].copy()

# remove whitespace from transit routes
hs_route_df["transit_routes"] = hs_route_df["transit_routes"].astype(str).str.replace(" ", "")

# split transit routes into list
hs_route_df["route"] = hs_route_df.transit_routes.str.split(",")

# add agency id column to hotspot route dataframe
agency_id_dict = {
    "AC Transit": "AC",
    "BART": "BA",
    "CityBus": "SR",
    "County Connection": "CC",
    # "Dixon Readi-Ride": "", # not in transit routes
    "FAST": "FS",
    "Golden Gate Transit": "GG",
    "LAVTA": "WH",
    "Marin Transit": "MA",
    "NVTA": "VN",
    "Petaluma Transit": "PE",
    "SamTrans": "SM",
    "SFMTA": "SF",
    "SolTrans": "ST",
    "Sonoma County Transit": "SO",
    "Tri Delta": "3D",
    "Union City Transit": "UC",
    "VTA": "SC",
    "WestCAT": "WC",
}
hs_route_df["agency_id"] = hs_route_df["agency"].map(agency_id_dict)

# explode the route column into multiple rows
hs_route_expode_df = hs_route_df[["hotspot_id", "agency", "agency_id", "route"]].explode("route")

In [47]:
hs_route_expode_df["route_id"] = hs_route_expode_df["agency_id"] + ":" + hs_route_expode_df["route"]

In [48]:
# update route_id for routes that have a different id in the transit routes table
route_dict = {
    "BA:Vine29": "VN:29",
    "BA:SolTransG": "ST:G",
    "BA:R": "ST:R",
    "BA:GGT580": "GG:580",
    "BA:704": "GG:704",
    "BA:AC72R": "AC:72R",
    "BA:72": "AC:72",
    "BA:72M": "AC:72M",
    "BA:800": "AC:800",
    "BA:7": "AC:7",
    "BA:76": "AC:76",
    "BA:376": "AC:376",
    "BA:L": "AC:L",
    "BA:WestCATJK": "WC:JX",
    "BA:JPX": "WC:JPX",
    "BA:JL": "WC:J",
    "BA:JR": "WC:J",
    "SR:1": "SR:01",
    "SR:2": "SR:02",
    "SR:2B": "SR:02B",
    "SR:3": "SR:03",
    "SR:5": "SR:05",
    "SR:4": "SR:04",
    "SR:4B": "SR:04B",
    "SR:6": "SR:06",
    "SR:8": "SR:08",
    "SR:9": "SR:09",
    "SR:20": "SO:20",
    "SR:30": "SO:30",
    "SR:34": "SO:34",
    "SR:46": "SO:46",
    "SR:44": "SO:44",
    "SR:48": "SO:48",
    "SR:60": "SO:60",
    "SR:62": "SO:62",
    "SR:72": "GG:72",
    # "SR:95": None,
    "SR:101": "GG:101",
    # "FS:microtransit": None,
    # "PE:10": None,
    # "PE:501": None,
    "SM:ECROwl": "SM:ECRO",
    "SM:296Owl": "SM:296O",
    # "SF:K": None,
    "SO:101(GGT)": "GG:101",
    # "SO:95": None,
    "UC:41": "AC:41",
    "UC:56": "AC:56",
    "UC:97": "AC:97",
    "UC:210": "AC:210",
    # "SC:500": None,
}
hs_route_expode_df["route_id"] = hs_route_expode_df["route_id"].replace(route_dict)

In [49]:
# merge the route id with the bus transit gdf
hs_route_gdf = pd.merge(
    bus_lr_gdf,
    hs_route_expode_df,
    on="route_id",
    how="right",
    suffixes=["_gtfs", "_hotspot"],
    indicator=True,
)

In [50]:
hs_route_gdf["_merge"].value_counts()

_merge
both          518881
right_only         7
left_only          0
Name: count, dtype: int64

In [51]:
hs_route_gdf.query("_merge == 'right_only'")["route_id"].unique()

array(['SO:46', 'GG:72', 'SR:95', nan, 'FS:microtransit', 'SO:95',
       'SC:500'], dtype=object)

In [52]:
# drop duplicate routes
sub_cols = [
    "hotspot_id",
    "route_id",
    "direction_id",
    "agency_id_gtfs",
    "route_type",
    "agency_name",
    "route_name",
    "route_type_name",
    "direction_name",
    "agency_url",
    "route",
]
hs_route_dedup = hs_route_gdf.query("_merge == 'both'").drop_duplicates(subset=sub_cols)

## Publish datasets to ArcGIS Online

In [53]:
import os

# checking if the directory demo_folder
# exist or not.
if not os.path.exists("Data"):
    # if the demo_folder directory is not present
    # then create it.
    os.makedirs("Data")

In [54]:
# export features to local directory
point_path = os.path.join("Data", "hs_point_gdf.geojson")
hs_point_gdf.to_file(point_path, driver="GeoJSON")

line_path = os.path.join("Data", "hs_line_gdf.geojson")
hs_line_gdf.to_file(line_path, driver="GeoJSON")

In [55]:
# publish point features to agol
publish_geojson_to_agol(
    geojson_path=point_path,
    layer_name="BusAID Hotspots - Point (December 2023)",
    layer_snippet="BusAID Hotspot Point Dataset",
    tags="mtc, bay area, busaid, bus, transit, hotspots",
    client=gis,
    folder="bus_aid",
    overwrite=True,
    f_layer_id="b03b3b392d324f2d828eaad56932e93b",
)

Overwrote hosted feature layer with id: b03b3b392d324f2d828eaad56932e93b


In [56]:
# publish line features to agol
publish_geojson_to_agol(
    geojson_path=line_path,
    layer_name="BusAID Hotspots - Line (December 2023)",
    layer_snippet="BusAID Hotspot Line Dataset",
    tags="mtc, bay area, busaid, bus, transit, hotspots",
    client=gis,
    folder="bus_aid",
    overwrite=True,
    f_layer_id="c31d2e1c793f43c68ad79cc544453fe9",
)

Overwrote hosted feature layer with id: c31d2e1c793f43c68ad79cc544453fe9


In [57]:
# drop merge
hs_route_dedup.drop(columns=["_merge"], inplace=True)

# export features to local directory
hs_route_path = os.path.join("Data", "hotspot_bus_routes_gdf.geojson")
hs_route_dedup.to_file(hs_route_path, driver="GeoJSON")

In [58]:
# publish routes near lines to agol
publish_geojson_to_agol(
    geojson_path=hs_route_path,
    layer_name="Hotspot Transit Routes",
    layer_snippet="This dataset represents transit routes that run through hotspots.",
    tags="mtc, bay area, busaid, bus, transit, hotspots",
    client=gis,
    folder="bus_aid",
    overwrite=True,
    f_layer_id="46749db760bc4371b79f7ae223755b15",
)

Overwrote hosted feature layer with id: 46749db760bc4371b79f7ae223755b15


## Spatial overlays

In [59]:
# overlay hotspot transit routes with epcs
hs_epc_gdf = gpd.sjoin(
    hs_route_dedup,
    epc_gdf[["geoid", "epc_2050", "epc_class", "geometry"]].to_crs("EPSG:26910"),
    how="left",
    predicate="intersects",
)

In [60]:
# create a dataframe of hotspots deduplicating by id, geoid, epc_2020, and epc_class
hs_epc_dedup = hs_epc_gdf[["hotspot_id", "geoid", "epc_2050", "epc_class"]].drop_duplicates(
    subset=["hotspot_id", "geoid", "epc_2050", "epc_class"]
)

In [61]:
# explode pdas
pda_explode_gdf = pda_gdf.explode()

In [62]:
# spatially join hotspot transit routes with pdas
hs_pda_gdf = gpd.sjoin(
    hs_route_dedup,
    pda_explode_gdf[["join_key", "pda_name", "geometry"]].to_crs("EPSG:26910"),
    how="left",
    predicate="intersects",
)

In [63]:
# Create a flag column to indicate if a hotspot is within a pda
hs_pda_gdf["pda_flag"] = np.where(hs_pda_gdf["join_key"].isnull(), 0, 1)

In [64]:
# create a dataframe of hotspots deduplicating by id, join_key, pda_name, and route_id
hs_pda_dedup = (
    hs_pda_gdf[["hotspot_id", "join_key", "pda_name", "route_id", "pda_flag"]]
    .sort_values("pda_flag", ascending=False)
    .drop_duplicates(subset=["hotspot_id", "route_id"])
)

## Summarize tracts by hotspot id, epc, and epc category

In [65]:
# summarize tracts by hotspot, epc_2050, and epc_class
hs_tract_summary = (
    hs_epc_dedup.groupby(["hotspot_id", "epc_2050", "epc_class"])
    .size()
    .reset_index()
    .rename(columns={0: "total_census_tracts"})
)

In [78]:
# Create new column and calculate percentage of total hotspot tracts
# hs_tract_summary["pct_hs_total_tracts"] =

hs_tract_summary["pct_total_hs_tracts"] = hs_tract_summary.groupby("hotspot_id")[
    "total_census_tracts"
].transform(lambda x: x / x.sum())

In [79]:
# write summary results to existing excel document
excel_path = os.path.join(dir, "BusAID Hotspot Master List_112823.xlsx")
with pd.ExcelWriter(excel_path, mode="a", if_sheet_exists="replace") as writer:
    hs_tract_summary.to_excel(writer, sheet_name="hotspot_epc_summary", index=False)

## Summarize transit routes by hotspot id, and whether they intersect with pdas

In [80]:
hs_pda_rt_summary = (
    hs_pda_dedup.groupby(["hotspot_id", "pda_flag"])
    .size()
    .reset_index()
    .rename(columns={0: "total_routes"})
)

In [81]:
# create new column and calculate the total number of routes that pass through pdas by the total number of routes that pass through hotspots
hs_pda_rt_summary["pct_total_routes"] = hs_pda_rt_summary.groupby("hotspot_id")[
    "total_routes"
].transform(lambda x: x / x.sum())

In [82]:
# write summary results to existing excel document
with pd.ExcelWriter(excel_path, mode="a", if_sheet_exists="replace") as writer:
    hs_pda_rt_summary.to_excel(writer, sheet_name="hotspot_pda_summary", index=False)

In [70]:
# test options for rows with only a single hotspot_id
pda_melt = hs_pda_rt_summary.melt(id_vars=["hotspot_id", "pda_flag"], value_vars=["total_routes", "pct_total_routes"])
pda_melt["new_variable"] = pda_melt["variable"] + "_" + pda_melt["pda_flag"].astype(str)
pda_melt.pivot(index="hotspot_id", columns="new_variable", values="value").reset_index()